# Library

In [ ]:
import pandas as pd
import numpy as np
import re
from tqdm import tqdm
tqdm.pandas()
# from CHAID import Tree
import ast

import optbinning
from optbinning import BinningProcess,ContinuousOptimalBinning
from optbinning import OptimalBinning

from datetime import datetime
from dateutil.relativedelta import relativedelta

# import pyreadstat


# train-test split

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

# Folder Paths

In [ ]:
# path_main=r"D:\Pranto\Working\LSA and Writeoff\Data\Updated\Final Updated V1"
month=r"MAY25"
path_main=r"D:\Pranto\Working\Collection Model\Collection New Model\3 to 3 Plus PV\Model Scoring\Base for Scoring"

path_binning=r"D:\Pranto\Working\Collection Model\Collection New Model\3 to 3 Plus PV\Final Model Files\Data\Binning Files"

names=rf"PV 3 to 3+ Scoring file with all variables_{month}"

binning_file_name=rf"3 TO 3 PLUS JAN24 TO MAR25_PV_MODEL_Overall Data Binning File"

# Numerical binning

### Numerical binning Data

In [ ]:

# main Data

df_ftu=pd.read_csv(rf"{path_main}\{names}.csv",encoding='latin-1')


# Numwrical Binning File
df_num_binning=pd.read_excel(rf"{path_binning}\{binning_file_name}.xlsx",sheet_name="Numerical Binning Data V1")

print("Main Dataset Shape",df_ftu.shape)
print("Binning Dataset Shape",df_num_binning.shape)


print(df_num_binning.columns)


df_ftu.head(3)

In [ ]:
data=df_ftu
binning_df=df_num_binning

# Change the inputs according to the binning file

In [ ]:
name = 'WOE'  # Columns name

# /////////////////////////////////////////////////////////////////////////////////////////
# Columns name according to Binning file(categorical,numerical)

Variable = 'Variable'  # Variable name
Bin = 'Bin'  # Bins
Group = 'WoE'  # Group Value
Bad_Rate = 'Event rate'

# /////////////////////////////////////////////////////////////////////////////////////////
# Target Variable name for checking nearest bad rate

target = None  # Set target to None to test dynamic handling
# target = 'TARGET_VARIABLE_V1'  # Uncomment to provide a target


# /////////////////////////////////////////////////////////////////////////////////////////

# Specify the method to use for unmapped values: "lowest", "highest", "average", or "nearest"
methods = "highest"  # Set the desired method here

# Apply binning for numerical columns

In [ ]:
# Clean binning file data_V1
def clean_data_V1frame(df):
    try:
        df = df[df['Group'] != 'Totals']
        # Convert Group to numeric for manipulation (Totals becomes NaN)
        df["Group"] = pd.to_numeric(df["Group"], errors="coerce")

        # Drop rows where Group is "Totals"
        df = df.dropna(subset=["Group"])

        # Drop rows where Bin is "Special" or "(-inf, inf)"
        df = df[~df["Bin"].isin(["Special", "(-inf, inf)"])]

        # Process "Missing" values
        mask_missing = (df["Bin"] == "Missing") & (df["Count"] > 0)
        df.loc[mask_missing, "Group"] = df.loc[mask_missing, "Group"] - 1  # Adjust Group

        # Drop other "Missing" values where Count is 0
        df = df[~((df["Bin"] == "Missing") & (df["Count"] == 0))]

        # Reset index
        df = df.reset_index(drop=True)

        return df
    except:
        pass

# Clean data_V1
binning_df=clean_data_V1frame(binning_df)



# Function to convert bin strings to tuples
def parse_bin(bin_str):
    if bin_str == 'Missing':
        return None
    bounds = bin_str.strip('()[]').split(',')
    return (float(bounds[0]), float(bounds[1]))

# Group by Variable and create dictionary of bin ranges with corresponding group values
bin_ranges = {}
bin_groups = {}
for variable, group_data in binning_df.groupby(Variable):
    bin_ranges[variable] = [parse_bin(bin_str) for bin_str in group_data[Bin]]
    bin_groups[variable] = group_data[Group].tolist()

# Function to map values to bins and retrieve corresponding group value
def map_to_bins(value, bins, groups):
    if pd.isna(value):
        return groups[-1]  # If value is missing, return the last group value
    for i, bin_range in enumerate(bins):
        if bin_range is not None and bin_range[0] <= value < bin_range[1]:
            return groups[i]
    return groups[-1]  # If value not found in any bin, return the last group value

# Apply binning and create new column_B with the last group value for missing values
for column in binning_df[Variable].unique():
    if f'{column}_{name}' not in data:  # Check if the column_B does not exist in the dataset
        bins = bin_ranges[column]
        groups = bin_groups[column]
        data[f'{column}_{name}'] = [map_to_bins(value, bins, groups) for value in data[column]]

# Create DataFrame with new column_B
df_binned = pd.DataFrame(data)

print(df_binned.shape)

df_binned.head(3)


In [ ]:
# df_binned.to_csv("abc_num_bin.csv",index=False)

# Categorical Binning

### Categorical binning dataframe

In [ ]:
df_ftu=df_ftu

# Categorical binning File
df_cat_bin=pd.read_excel(rf"{path_binning}\{binning_file_name}.xlsx",sheet_name="Categorical Binning Data V1")

print("Main Dataset Shape",df_ftu.shape)
print("Binning Dataset Shape",df_cat_bin.shape)


print(df_cat_bin.columns)


df_ftu.head(3)


In [ ]:

data=df_ftu

binning_df=df_cat_bin

# Categorical Binning Apply

In [ ]:

# Clean binning file data_V1
def clean_data_V1frame(df):
    try:
        df = df[df['Group'] != 'Totals']
        # Convert Group to numeric for manipulation (Totals becomes NaN)
        df["Group"] = pd.to_numeric(df["Group"], errors="coerce")

        # Drop rows where Group is "Totals"
        df = df.dropna(subset=["Group"])

        # Drop rows where Bin is "Special" or "(-inf, inf)"
        df = df[~df["Bin"].isin(["Special", "(-inf, inf)"])]

        # Process "Missing" values
        mask_missing = (df["Bin"] == "Missing") & (df["Count"] > 0)
        df.loc[mask_missing, "Group"] = df.loc[mask_missing, "Group"] - 1  # Adjust Group

        # Drop other "Missing" values where Count is 0
        df = df[~((df["Bin"] == "Missing") & (df["Count"] == 0))]

        # Reset index
        df = df.reset_index(drop=True)

        return df
    except:
        pass

# Clean data_V1
binning_df=clean_data_V1frame(binning_df)






# Function to map values to bins and retrieve corresponding group value
def map_to_bins_categorical(value, bins, groups):
    if pd.isna(value):
        return groups[-1]
        # return groups[-1] if groups else None  # Handle missing group gracefully
    for i, bin_str in enumerate(bins):
        bin_values = [s for s in bin_str.strip("[]").split("'") if s]
        bin_values1 = [int(x) if x.isdigit() else x for x in bin_values]
        if isinstance(value, str):
            if value in bin_values:
                return groups[i]
        else:
            if value in bin_values1:
                return groups[i]
    return None  # If value not found in any bin, return None

# ////////////////////////////////////////////////////////////////////////////////////////////////////////////

# Function to compute bad rate for each category in a column
def compute_bad_rate(df, column, target):
    bad_rate_df = df.groupby(column)[target].mean().reset_index()
    bad_rate_df.columns = [column, Bad_Rate]  # Rename column for consistency
    return bad_rate_df

# ////////////////////////////////////////////////////////////////////////////////////////////////////////////

def find_nearest_group(bad_rate, binning_df, variable, method):
    variable_binning = binning_df[binning_df[Variable] == variable]
    if method == "nearest":
        # Find the nearest bad rate
        nearest_group = variable_binning.loc[(variable_binning[Bad_Rate] - bad_rate).abs().idxmin()]
    elif method == "highest":
        # Find the group with the highest bad rate
        nearest_group = variable_binning.loc[variable_binning[Bad_Rate].idxmax()]
    elif method == "lowest":
        # Find the group with the lowest bad rate
        nearest_group = variable_binning.loc[variable_binning[Bad_Rate].idxmin()]
    elif method == "average":
        # Find the group closest to the average bad rate
        average_bad_rate = variable_binning[Bad_Rate].mean()
        nearest_group = variable_binning.loc[(variable_binning[Bad_Rate] - average_bad_rate).abs().idxmin()]
    
    # Ensure `nearest_group` is a Series before accessing `Group`
    return nearest_group[Group]



# ////////////////////////////////////////////////////////////////////////////////////////////////////////////

unmapped_details = []

# Process each variable in the binning dataframe
for column in tqdm(binning_df[Variable].unique(),desc="Processing Time = "):
    # Extract bins and groups for the current column
    bins = binning_df[binning_df[Variable] == column][Bin].tolist()
    groups = binning_df[binning_df[Variable] == column][Group].tolist()

    # Map values to bins
    data[f"{column}_{name}"] = [map_to_bins_categorical(value, bins, groups) for value in data[column]]

    # Compute bad rates only if target is not None
    bad_rate_df = None
    if target is not None:
        bad_rate_df = compute_bad_rate(data, column, target)

    # Handle unmapped values
    unique_values = data[column].unique()
    for value in unique_values:
        if data.loc[data[column] == value, f"{column}_{name}"].isna().all():
            bad_rate = None
            nearest_group = None

            # If target exists, calculate the nearest group based on the bad rate
            if bad_rate_df is not None:
                bad_rate_row = bad_rate_df.loc[bad_rate_df[column] == value, Bad_Rate]
                if not bad_rate_row.empty:
                    bad_rate = bad_rate_row.values[0]
                    nearest_group = find_nearest_group(bad_rate, binning_df, column, methods)
            else:
                # Use the pre-selected method if no target is available
                nearest_group = find_nearest_group(None, binning_df, column, methods)

            # Log details of unmapped values
            unmapped_details.append({
                "Column": column,
                "Unmapped_Value": value,
                "Method": methods,
                "Assigned_Bad_Rate": bad_rate,
                "Assigned_Group": nearest_group
            })

            # Map the unmapped value to the determined group
            data.loc[data[column] == value, f"{column}_{name}"] = nearest_group


            # Update the binning_df with the new bin
            existing_bin_rows = binning_df[(binning_df[Variable] == column) & (binning_df[Group] == nearest_group)]

            new_bin_value = f"[{value}]"

            if not existing_bin_rows.empty:
                # Update existing bin with the new value
                existing_bins = existing_bin_rows.iloc[0][Bin]

                # Parse the existing bin into a list (removing brackets and splitting by commas)
                existing_bin_list = existing_bins.strip("[]").split(",")

                # Ensure all elements are strings to avoid type errors
                existing_bin_list = [str(item).strip() for item in existing_bin_list if item]

                # Append the new value if it doesn't already exist
                if str(value) not in existing_bin_list:
                    existing_bin_list.append(str(value))

                    # Sort the list and reconstruct it into the exact required format
                    sorted_bin_list = sorted(existing_bin_list)
                    updated_bin_V1 = "['" + "' '".join(sorted_bin_list) + "']"
                    updated_bin_V2 = updated_bin_V1.replace("\n", "").replace("''", "'").replace(", ", " ").replace("]", "").replace("[", "['").strip() + "]"
                    updated_bin_V3 = re.sub(r"\['+'", "['", updated_bin_V2)

                    # Update only the rows where the value was added
                    binning_df.loc[existing_bin_rows.index, Bin] = updated_bin_V3
            else:
                # Add a new row if no bin exists
                updated_bin_V1 = f"['{value}']"  # Create the initial bin in the required format
                # binning_df = binning_df.append({Variable: column, Bin: updated_bin_V1}, ignore_index=True)
                binning_df = pd.concat([binning_df, pd.DataFrame([{Variable: column, Bin: updated_bin_V1}])], ignore_index=True)

# ////////////////////////////////////////////////////////////////////////////////////////////////////////////

# Convert unmapped details to a DataFrame
unmapped_df = pd.DataFrame(unmapped_details)

# ////////////////////////////////////////////////////////////////////////////////////////////////////////////

# Display the updated DataFrame
print("Updated DataFrame:")
print(data.shape)

# Updated binning_df with new bins for unmapped values
print("\nUpdated Binning DataFrame:")
print(binning_df.shape)

# Display the unmapped values DataFrame
print("\nUnmapped Details DataFrame:")
unmapped_df.head(5)


In [ ]:
unmapped_df

In [ ]:
data.isna().sum().to_dict()

# Export Binned Data

In [ ]:
data.to_csv(rf"{path_main}\{names}_apply_binning V1.csv",index=False)

In [ ]:
with pd.ExcelWriter(rf"{path_main}\{names}_biining file with highest bad rate.xlsx",engine='xlsxwriter') as want:
    binning_df.to_excel(want,sheet_name="Categorical Binning",index=False)
    unmapped_df.to_excel(want,sheet_name="Unmapped Values",index=False)
# binning_df.to_excel(rf"{path_main}\{names}_biining file with highest bad rate.xlsx",index=False)